In [1]:
import tensorflow as tf
import seaborn as sns
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler ,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score, mean_squared_error,mean_absolute_percentage_error
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM,Dropout, BatchNormalization, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from keras.models import Sequential
from keras.optimizer_v2.adam import Adam
from keras.optimizer_v2.rmsprop import RMSprop
import talib

In [2]:
# Get the data for the SPY ETF by specifying the stock ticker, start date, and end date
historical_data = yf.download('0005.hk')
historical_data.shape

[*********************100%***********************]  1 of 1 completed


(5490, 6)

In [3]:
train_df,test_df = historical_data[1:5000], historical_data[5000:] 

In [4]:
train = train_df
scalers={}
for i in train_df.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_df
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

/var/folders/8h/w0jn89bj2w5g85gj4ck7w0t00000gn/T/ipykernel_19578/3151536076.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
/var/folders/8h/w0jn89bj2w5g85gj4ck7w0t00000gn/T/ipykernel_19578/3151536076.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[i]=s_s
/var/folders/8h/w0jn89bj2w5g85gj4ck7w0t00000gn/T/ipykernel_19578/3151536076.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [5]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
    X, y = list(), list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
    # slicing the past and future parts of the window
#         print(series)
        past, future = series[window_start:past_end, :], series[past_end:future_end, :]
        X.append(past)
#         print(future)
        y.append(future)
    return np.array(X), np.array(y)

In [6]:
n_past = 10
n_future = 5 
n_features = 6

In [7]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))[:,:,3]
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], 1))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))[:,:,3]
y_test=y_test.reshape(y_test.shape[0],y_test.shape[1],1)
print(y_train.shape)
print(y_test.shape)

(4985, 5, 1)
(476, 5, 1)


In [8]:
# train the model
def build_model():
	# prepare data
# 	train_x, train_y = to_supervised(train, n_input)
	# define parameters
	verbose, epochs, batch_size = 0, 500, 4
	n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
	# define model
	model = Sequential()
	model.add(LSTM(1, activation='relu', input_shape=(n_past, n_features)))
# 	model.add(Dense(100, activation='relu'))
# 	model.add(Dense(n_outputs))
	model.compile(loss='mse', optimizer='adam')
	# fit network
	model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model
 
# make a forecast
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, 0]
	# reshape into [1, n_input, 1]
	input_x = input_x.reshape((1, len(input_x), 1))
	# forecast the next week
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat
 
# evaluate a single model
def evaluate_model(train, test, n_input):
	# fit model
	model = build_model(train, n_input)
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = forecast(model, history, n_input)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	# evaluate predictions days for each week
	predictions = array(predictions)
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores

In [9]:
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

Metal device set to: Apple M1 Pro


2021-12-14 17:08:17.384572: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-14 17:08:17.384699: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10, 6)]      0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 100),        42800       ['input_1[0][0]']                
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                                  
 repeat_vector (RepeatVector)   (None, 5, 100)       0           ['lstm[0][0]']                   
                                                                                              

In [10]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=1,callbacks=[reduce_lr])
# model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
# history_e2d2=model_e2d2.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=0,callbacks=[reduce_lr])

Epoch 1/25


2021-12-14 17:08:17.607459: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-14 17:08:18.290539: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-14 17:08:18.498395: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-14 17:08:18.551939: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-14 17:08:18.652425: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


  9/156 [>.............................] - ETA: 2s - loss: 0.0569 

2021-12-14 17:08:18.719305: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


156/156 [==============================] - ETA: 0s - loss: 0.0054

2021-12-14 17:08:21.358794: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-14 17:08:21.433662: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-14 17:08:21.465307: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


156/156 [==============================] - 4s 18ms/step - loss: 0.0054 - val_loss: 7.0204e-04 - lr: 0.0010
Epoch 2/25
156/156 [==============================] - 3s 17ms/step - loss: 9.9219e-04 - val_loss: 6.6892e-04 - lr: 9.0000e-04
Epoch 3/25
156/156 [==============================] - 3s 16ms/step - loss: 8.8371e-04 - val_loss: 0.0021 - lr: 8.1000e-04
Epoch 4/25
156/156 [==============================] - 2s 16ms/step - loss: 9.4721e-04 - val_loss: 0.0032 - lr: 7.2900e-04
Epoch 5/25
156/156 [==============================] - 3s 16ms/step - loss: 8.4266e-04 - val_loss: 5.3121e-04 - lr: 6.5610e-04
Epoch 6/25
156/156 [==============================] - 3s 16ms/step - loss: 8.3923e-04 - val_loss: 5.4110e-04 - lr: 5.9049e-04
Epoch 7/25
156/156 [==============================] - 2s 16ms/step - loss: 8.3021e-04 - val_loss: 7.4553e-04 - lr: 5.3144e-04
Epoch 8/25
156/156 [==============================] - 3s 16ms/step - loss: 8.5859e-04 - val_loss: 6.8639e-04 - lr: 4.7830e-04
Epoch 9/25
156/156 

In [11]:
pred_e1d1=model_e1d1.predict(X_test)
pred_e1d1.shape

2021-12-14 17:09:25.498302: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-14 17:09:25.588511: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-12-14 17:09:25.629849: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


(476, 5, 6)

In [12]:

    scaler = scalers['scaler_'+'Close']
#     pred1_xe1d1[:,:,index]=scaler.inverse_transform(pred1_e1d1[:,:,index])
    pred_e1d1[:,:,0]=scaler.inverse_transform(pred_e1d1[:,:,0])
#     pred1_e2d2[:,:,index]=scaler.inverse_transform(pred1_e2d2[:,:,index])
#     pred_e2d2[:,:,index]=scaler.inverse_transform(pred_e2d2[:,:,index])
    y_train[:,:,0]=scaler.inverse_transform(y_train[:,:,0])
    y_test[:,:,0]=scaler.inverse_transform(y_test[:,:,0])

In [13]:
from sklearn.metrics import mean_absolute_error
for j in range(0,6):
    print("Day ",j,":")
    print("MAE-E1D1 : ",mean_absolute_error(y_test[:,:,0],pred_e1d1[:,:,0]),end=", ")
#         print("MAE-E2D2 : ",mean_absolute_error(y_test[:,j-1,index],pred1_e2d2[:,j-1,index]))
    print()
#     print()

Day  0 :
MAE-E1D1 :  1.517189725507207, 
Day  1 :
MAE-E1D1 :  1.517189725507207, 
Day  2 :
MAE-E1D1 :  1.517189725507207, 
Day  3 :
MAE-E1D1 :  1.517189725507207, 
Day  4 :
MAE-E1D1 :  1.517189725507207, 
Day  5 :
MAE-E1D1 :  1.517189725507207, 


In [19]:
pred_e1d1[:,:,0]

array([[60.283524, 60.070297, 59.91407 , 59.897537, 59.945637],
       [60.052162, 59.84348 , 59.69194 , 59.680466, 59.733456],
       [59.51294 , 59.24235 , 59.069927, 59.056587, 59.11564 ],
       ...,
       [44.92573 , 44.776684, 44.983494, 45.364304, 45.782223],
       [45.190926, 45.05243 , 45.254967, 45.627384, 46.036568],
       [45.696526, 45.630978, 45.859108, 46.23417 , 46.63684 ]],
      dtype=float32)

In [15]:
y_test[:,:,0]

array([[59.90000153, 59.29999924, 59.84999847, 59.84999847, 59.95000076],
       [59.29999924, 59.84999847, 59.84999847, 59.95000076, 59.75      ],
       [59.84999847, 59.84999847, 59.95000076, 59.75      , 59.79999924],
       ...,
       [44.84999847, 45.65000153, 45.5       , 45.29999924, 45.20000076],
       [45.65000153, 45.5       , 45.29999924, 45.20000076, 45.09999847],
       [45.5       , 45.29999924, 45.20000076, 45.09999847, 44.5       ]])

In [16]:
from sklearn.metrics import r2_score
r2_score(pred_e1d1[:,:,0],y_test[:,:,0])

0.8940007428641907

In [17]:
newData = yf.download('0005.hk')
print(newData.tail())
newData[-5:]['Close'].values

[*********************100%***********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2021-12-08  45.950001  45.950001  45.099998  45.500000  45.500000   8590654
2021-12-09  45.450001  45.900002  45.150002  45.299999  45.299999   6908033
2021-12-10  45.099998  45.400002  44.950001  45.200001  45.200001   6090109
2021-12-13  45.200001  45.250000  44.500000  45.099998  45.099998  10861423
2021-12-14  44.150002  44.599998  44.150002  44.500000  44.500000   7998591


array([45.5       , 45.29999924, 45.20000076, 45.09999847, 44.5       ])

In [18]:
pred_e1d1[:,:,0][-1]

array([45.696526, 45.630978, 45.859108, 46.23417 , 46.63684 ],
      dtype=float32)